In [24]:
# from google.colab import drive
# drive.mount('/content/drive')

In [25]:
# [필수] 파일 위치 재설정
# %cd ./drive/MyDrive/PreProcess_1

In [26]:
# !pwd

In [27]:
# 전처리 1-3, 앵커 목소리가 기자 목소리로 전환될 때 timestamp 남기기
# (** 이전과 다르게 코랩에 최적화된 형태로 코드를 정리하였습니다. 참고 부탁드립니다.)

# [ 필수 공지 ]
# GitHub에 함께 첨부한 'database.yml' 파일 다운 후에 해당 코드를 돌려주세요.
# 'File_selection.ipynb' 파일을 다운받은 후에 import하시면 더 편하게 처리 가능합니다. 

# 참고 자료 : https://github.com/pyannote/pyannote-audio/tree/master/tutorials/pretrained/model

# [ 고민 지점 ]
# 연산 처리 시간이 생각보다 오래 걸리는데 이걸 어떻게 극복해야 할까? 

# [ 고민 해결 방식 ]  
# 2분 이하의 동영상만 사용하면 연산 속도를 줄일 수 있음.
# 로컬 주피터노트북에서 파일 2개 처리할 때 1분 걸리는 것을 코랩에서는 12초만에 처리 가능함.

# [ 정리 ]
# "2분 초과 영상 제외 + 코랩 GPU 사용"으로 해결 가능

In [28]:
# pyannote 다운받을 수 있는 코드
# !pip install git+https://github.com/pyannote/pyannote-audio

In [29]:
# !pip install import_ipynb

In [30]:
import torch
import pyannote
from pyannote.database import get_protocol
from pyannote.database import FileFinder
from pyannote.audio.utils.signal import Binarize

import re
import pandas as pd

# 1-1 전처리 (File_selection.ipynb)에서 작업한 함수 가져오기
import import_ipynb
from File_selection import *

In [31]:
# 코드 실행시간 측정
start = time.time()

In [32]:
Whole_MOV = Whole_MOV_List('./MOV')
Under_2min_MOV = Two_min_under_MOV(Whole_MOV)
Using_MOV_List, Origin_file_List = one_anchor_starts_MOV(Under_2min_MOV)

Except MOV_file name : ./MOV/readnaversid1750oid448aid0000195268.mp4



In [33]:
# MOV_to_WAV 함수 : 영상(.mp4) -> 음성(.wav) 전환 작업

def MOV_to_WAV(MOV_List):
    WAV_file = np.array([])
    
    for MOV in notebook.tqdm(MOV_List): 
        clip = VideoFileClip(MOV)

        # 파일 저장 위치 변경
        sound_file_name = re.sub(pattern='MOV', repl='WAV', string=str(MOV))
        # 파일 저장 형태 변경
        sound_file_name = re.sub(pattern='mp4', repl='wav', string=sound_file_name)
        
        clip.audio.write_audiofile(sound_file_name)

In [34]:
# WAV_file_name 함수 : WAV 파일 array 반환
def Whole_WAV_List(WAV_Directory):
    WAV_file = os.listdir(WAV_Directory)
    WAV_List = np.array([WAV_Directory + '/' + WAV for WAV in WAV_file])
    
    return WAV_List

In [35]:
# Using_sad 함수 : Speech activity detection (sad) 사전훈련 모델 사용 선언 및 설정
def Using_sad(WAV_List):
    sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
    preprocessors = {'audio': FileFinder()}
    protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset',
                            preprocessors=preprocessors)
    
    # Speech activity detection 환경 설정
    binarize = Binarize(offset=0.52, onset=0.52, log_scale=True,
                       min_duration_off=0.1, min_duration_on=0.1)

    # 앵커 -> 기자 넘어가는 시점 계산
    test_file = {}
    results = pd.DataFrame({'WAV_name':[], 'TimeStamp':[]})
    
    for k in notebook.tqdm(WAV_List):
        test_file['audio'] = k
        sad_scores = sad(test_file)
        speech = binarize.apply(sad_scores, dimension=1)
    
        # TimeStamp 편집 과정
        a = str(speech[0])
        a = re.sub(pattern='\-\-\>', repl='', string=a)
        a = re.sub(pattern='\[', repl='', string=a)
        a = re.sub(pattern='\]', repl='', string=a)
        a = str.split(a)
        
        b = a[1]
        b = datetime.datetime.strptime(b, '%H:%M:%S.%f').time()
        
        result = pd.DataFrame({'WAV_name':[test_file['audio']], 'TimeStamp':[b]})
        results = pd.concat([results, result])
    
    results = results.reset_index()
    
    return results

In [36]:
# Switching_moment 함수 : 전처리 1-3 파트 총망라
def Switching_moment(WAV_Directory):
    
    # wav 파일 이름 구하기
    WAV_files = Whole_WAV_List(WAV_Directory)
    
    # 앵커 -> 기자 넘어가는 시점 계산
    result_table = Using_sad(WAV_files)
    
    return result_table

In [37]:
# mp4 -> wav 파일 변환
# 이미 wav 파일로 변환 완료된 상태라면 넘어가도 무방함.
MOV_to_WAV(Using_MOV_List)

[MoviePy] Writing audio in ./WAV/readnaversid1750oid448aid0000195576.wav


100%|██████████| 1677/1677 [00:00<00:00, 2248.84it/s]

[MoviePy] Done.


[MoviePy] Writing audio in ./WAV/readnaversid1750oid448aid0000310334.wav


100%|██████████| 2057/2057 [00:01<00:00, 1926.82it/s]

[MoviePy] Done.



In [38]:
Switching_moment('./WAV')

Using cache found in /root/.cache/torch/hub/pyannote_pyannote-audio_master


,index,WAV_name,TimeStamp
0,0,./WAV/readnaversid1750oid448aid0000195576.wav,00:00:13.160000
1,0,./WAV/readnaversid1750oid448aid0000310334.wav,00:00:17.906000


In [39]:
# 실행시간 결과
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

0:00:12
